In [24]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

### Constants

In [25]:
# Change this to your location 
import variables
PATH = variables.stocks

In [26]:
# Start end date defaults
S_DATE = "2017-02-01"
E_DATE = "2022-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

### Get Column Data From CSVs

In [5]:
def get_column_from_csv(file, col_name):
    try:
        df = pd.read_csv(file)
    except FileNotFoundError:
        print("File does not exist")
    else:
        return df[col_name]


### Get Stock Tickers

In [6]:
tickers = get_column_from_csv(variables.wilshire, "Ticker")
print(len(tickers))

3481


### Save Stock Data to CSV

In [7]:
# Function to get a dataframe by providing a ticker and starting data
def save_to_csv_from_yahoo(folder, ticker):
    stock = yf.Ticker(ticker)

    try:
        print("Get Data for: ", ticker)
        # Get historical closing price data 
        df = stock.history(period="5y")

        # Wait  2 seconds
        time.sleep(2)

        # Remove the period for saving the file name
        # Save data to a CSV file 
        # File to save to 
        the_file = folder + ticker.replace(".", "_") + '.csv'
        print(the_file, " Saved")
        df.to_csv(the_file)

    except Exception as ex:
        print("Couldn't Get Data :", ticker)
         

### Download All Stocks

In [11]:
for x in range(0, 3481):
    save_to_csv_from_yahoo(PATH, tickers[x])
    print(f"{tickers[x]} downloaded")

Get Data for:  A
/Users/sammurray/Documents/stonks/A.csv  Saved
A downloaded
Get Data for:  AA
/Users/sammurray/Documents/stonks/AA.csv  Saved
AA downloaded
Get Data for:  AAL
/Users/sammurray/Documents/stonks/AAL.csv  Saved
AAL downloaded
Get Data for:  AAME
/Users/sammurray/Documents/stonks/AAME.csv  Saved
AAME downloaded
Get Data for:  AAN
/Users/sammurray/Documents/stonks/AAN.csv  Saved
AAN downloaded
Get Data for:  AAOI
/Users/sammurray/Documents/stonks/AAOI.csv  Saved
AAOI downloaded
Get Data for:  AAON
/Users/sammurray/Documents/stonks/AAON.csv  Saved
AAON downloaded
Get Data for:  AAP
/Users/sammurray/Documents/stonks/AAP.csv  Saved
AAP downloaded
Get Data for:  AAPL
/Users/sammurray/Documents/stonks/AAPL.csv  Saved
AAPL downloaded
Get Data for:  AAT
/Users/sammurray/Documents/stonks/AAT.csv  Saved
AAT downloaded
Get Data for:  AAWW
/Users/sammurray/Documents/stonks/AAWW.csv  Saved
AAWW downloaded
Get Data for:  AAXN
- AAXN: No data found, symbol may be delisted
/Users/sammurra

### Get Dataframe from CSV

In [8]:
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + ".csv", index_col=0)
    except FileNotFoundError:
        print("File is not here" )
        print('Expected file: ' + PATH + ticker + ".csv")
    else: 
        return df 
         

### Get all stocks downloaded in list

In [9]:
files = [x for x in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
# tickers.remove('.ds_Store')

tickers.sort()
len(tickers)

3261

### Add Daily Returns

In [10]:
def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close']/df['Close'].shift(1))-1
    return df

### Add Cumulative Return

In [11]:
def add_cum_return_to_df(df):
    df['daily_return'] = (1 + df['daily_return']).cumprod()
    return df

### Add Bollinger Bands

In [12]:
def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    return df

### Ichimokuk data to dataframe

In [13]:
def add_ichimoku(df):
     # Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 sessions)
     high_val = df['High'].rolling(window=9).max()
     low_val = df['Low'].rolling(window=9).min()
     df['Conversion'] = (high_val + low_val)/2

     # Base Line = (Highest Value in period + Lowest value in period)/2 (26 sessions)
     high_val2 = df['High'].rolling(window=26).max()
     low_val2 = df['Low'].rolling(window=26).min()
     df['Baseline'] = (high_val2 + low_val2)/2

     # Span A = (Conversion Value + Base Value)/2 (26 sessions)
     df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)

     # Span B = (Conversion Value + Base Value)/2 (26 sessions)
     high_val3 = df['High'].rolling(window=52).max()
     low_val3 = df['Low'].rolling(window=52).min()
     df['SpanB'] = ((high_val3 / low_val3)/2).shift(26)
     
     # Lagging Span = Price shifted back 26 periods
     df['Lagging'] = df['Close'].shift(-26)

     return df

### Add Daily, Cumulative Bollinger Bands and Ichimoku to dataframes

In [14]:
# Try for just one stock to make sure it works 
# try:
#     print("Working on :", "A")
#     new_df = get_stock_df_from_csv("A")
#     new_df = add_daily_return_to_df(new_df)
#     new_df = add_cum_return_to_df(new_df)
#     new_df = add_bollinger_bands(new_df)
#     new_df = add_ichimoku(new_df)
#     new_df.to_csv(PATH + 'A' + '.csv')

# except Exception as ex:
#     print(ex)

Workking on : A


In [20]:
count = 0
for ticker in tickers:
    count += 1
    try:
        print("Working on :", ticker)
        new_df = get_stock_df_from_csv(ticker)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cum_return_to_df(new_df)
        new_df = add_bollinger_bands(new_df)
        new_df = add_ichimoku(new_df)
        new_df.to_csv(PATH + ticker + '.csv')

    except Exception as ex:
        print(ex)
    
    print(f'{count}/{len(tickers)}')

Working on : A
1/3261
Working on : AA
2/3261
Working on : AAL
3/3261
Working on : AAME
4/3261
Working on : AAN
5/3261
Working on : AAOI
6/3261
Working on : AAON
7/3261
Working on : AAP
8/3261
Working on : AAPL
9/3261
Working on : AAT
10/3261
Working on : AAWW
11/3261
Working on : AAXN
12/3261
Working on : ABBV
13/3261
Working on : ABC
14/3261
Working on : ABCB
15/3261
Working on : ABEO
16/3261
Working on : ABG
17/3261
Working on : ABIO
18/3261
Working on : ABM
19/3261
Working on : ABMD
20/3261
Working on : ABR
21/3261
Working on : ABT
22/3261
Working on : ABTX
23/3261
Working on : AC
24/3261
Working on : ACA
25/3261
Working on : ACAD
26/3261
Working on : ACBI
27/3261
Working on : ACC
28/3261
Working on : ACCO
29/3261
Working on : ACER
30/3261
Working on : ACGL
31/3261
Working on : ACHC
32/3261
Working on : ACHV
33/3261
Working on : ACIA
34/3261
Working on : ACIW
35/3261
Working on : ACLS
36/3261
Working on : ACM
37/3261
Working on : ACMR
38/3261
Working on : ACN
39/3261
Working on : AC

### Plot with Bollinger Bands

In [37]:
def plot_with_boll_bands(df, ticker):
    fig = go.Figure()

    candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], 
    low=df['Low'], close=df['Close'], name='Candlestick')

    upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
    line=dict(color='rgba(250, 0, 0, 0.75)', 
    width=1), name='Upper Band')

    mid_line = go.Scatter(x=df.index, y=df['middle_band'], 
    line=dict(color='rgba(0, 0, 250, 0.75)', 
    width=1), name='Middle Band')

    lower_line = go.Scatter(x=df.index, y=df['lower_band'], 
    line=dict(color='rgba(0, 250, 0, 0.75)', 
    width=1), name='Lower Band')

    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(mid_line)
    fig.add_trace(lower_line)

    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")

    fig.update_layout(title=ticker + " Bollinger Bands", 
    height=800, width=1700, showlegend=True)
    fig.show()


    



    

### Plot Ichimoku

In [ ]:
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0, 250, 0, 0.4)'
    else:
        return 'rgba(250, 0, 0, 0.4)'

### Plots

In [39]:
test_df = get_stock_df_from_csv("PFE")
plot_with_boll_bands(test_df, "PFE")